# Preliminaries

In [5]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  8 22:29:59 2021

@author: ptbowen
"""
import sys
sys.path.append('..')
import worx.ElectromagneticObjects as emag
import worx.MathObjects as math
import worx.Geometries as geom
import worx.PlotTools as PlotTools

import numpy as np
from numpy.lib.scimath import sqrt
from mayavi import mlab
import copy

pi=np.pi
c=2.998e8
(xhat,yhat,zhat)=math.CreateEuclideanBasis()
vec_space=[xhat,yhat,zhat]

# Single modulated waveguide

First, we will implement a single modulated waveguide before creating the array, and plot the results.  All of this content you will have hopefully seen before now.

In [6]:
# Operating frequency and beam definition
f0=9.8e9
lmbda=c/f0
k=2*pi/lmbda
theta_b=90*pi/180
phi_b=0.1*pi/180
k_b=math.SphericalBasis('r')(theta_b,phi_b)*k
E_b=math.SphericalBasis('theta')(theta_b,phi_b)

# Waveguide design
n_wg=sqrt(3.55*(1-0.0027j))
pitch=0.008
N=5
L=N*pitch
a=14e-3
b=0.762e-3
WG=geom.Waveguide(a,b,n_wg,L)
WG.N_modes=1

# Create a half space for far-field calcs
HS=geom.HalfSpace()

# Data files for polarizability extraction
S11MagFile='../HFSS/S11mag_MikeCell_CapSweep.csv'
S11PhaseFile='../HFSS/S11phase_MikeCell_CapSweep.csv'
S21MagFile='../HFSS/S21mag_MikeCell_CapSweep.csv'
S21PhaseFile='../HFSS/S21phase_MikeCell_CapSweep.csv'
HFSS_Files={'S11Mag':S11MagFile,'S11Phase':S11PhaseFile,
            'S21Mag':S21MagFile,'S21Phase':S21PhaseFile}

# Create initial dipole.  This time it's parameterized.
r0=xhat*1.6e-3
dip=emag.Dipole(r0)
dip.extract(WG,HFSS_Files,parameterized=1)
f=np.linspace(8e9,12e9,200)
mod_range=np.linspace(0,1,5)
# dip.analyze(f=f,mod=mod_range)

# Dipole array design
z_pos=np.arange(pitch/2,(N+1/2)*pitch,pitch)+1.2e-3
x_pos=np.ones(np.shape(z_pos))*dip.r0[xhat]
y_pos=np.ones(np.shape(z_pos))*dip.r0[yhat]
r_pos=math.GridToListOfVectors({xhat:x_pos,yhat:y_pos,zhat:z_pos})
dip_array=dip.Array(r_pos)
for dipole in dip_array:
    WG.add_dipole(dipole)
    HS.add_dipole(dipole)
    
# Now, position it where you want it in space
R1=math.RotTensor(pi/2,'x',vec_space)
R2=math.RotTensor(-pi/2,'z',vec_space)
WG.Rotate(R1)
WG.Rotate(R2)
WG.Translate(yhat*(a/2))

# Define source and compute unperturbed feed wave
WG.Source(f0)
WG.Modulate(k_b,E_b)
# for dipole in WG.DipoleList: dipole.tune(f0,0.5)
WG.Compute(f0,compute_type='coupled')
HS.ComputeFarField(f0)

# Plot
mlab.init_notebook()
mlab.clf()
WG.MayaviPlot(1,plot_fields=True,field_clip=30)
PlotTools.MayaviAxes(10,-L/2,L/2,0,1.5*L,-L/10,L/2)
mlab.figure(figure=1,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [ ]:
# Find total far-field and plot
directivity=HS.FarField.directivity
k_ff=HS.FarField.k
(theta_grid,phi_grid,th_axis,ph_axis,dth,dph)=math.HalfSphereGrid(HS.FarField.Nth,HS.FarField.Nph)
Dir_dB=10*np.log10(np.real(directivity(k_ff(theta_grid,phi_grid)))+1e-16) 
Dir_dB[Dir_dB<0]=0
rhat=math.SphericalBasis('r')
X=Dir_dB*(rhat.dot(xhat))(theta_grid,phi_grid)
Y=Dir_dB*(rhat.dot(yhat))(theta_grid,phi_grid)
Z=Dir_dB*(rhat.dot(zhat))(theta_grid,phi_grid)

mlab.init_notebook()
mlab.clf()
mlab.mesh(X,Y,Z)
coordmax=np.max([X,Y,Z])
PlotTools.MayaviAxes(10,-coordmax,coordmax,-coordmax,coordmax,0,coordmax)
mlab.figure(figure=2,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

# Multi-waveguide array

Now we will create an array of waveguides, modulate them, and solve for both near and far fields.

In [8]:
# First, make a fresh half space so the old waveguide's dipoles aren't included.
HS=geom.HalfSpace()

# Copy waveguide to make array
WG_spacing=16e-3
N_wg=8
WG_Array=[0 for i in range(0,N_wg)]
for i in range(0,N_wg):
    angle=i*2*pi/N_wg
    translate_vec=(xhat*np.cos(angle)+yhat*np.sin(angle))*WG_spacing
    R=math.RotTensor(i*2*pi/N_wg,'z',vec_space)
    WG_Array[i]=copy.deepcopy(WG)
    WG_Array[i].Rotate(R)
    WG_Array[i].Translate(translate_vec)
    # You have to make sure all the dipoles are also added to the half space for the far field calculation.
    for dipole in WG_Array[i].DipoleList:
        dipole.tune(f0,0.3)
        HS.add_dipole(dipole)

# Plot geometry
mlab.figure(figure=1,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))
mlab.clf()
for wg in WG_Array:
    wg.MayaviPlot(1,plot_fields=False,field_clip=30)
PlotTools.MayaviAxes(10,-1.5*L,1.5*L,-1.5*L,1.5*L,-L/10,L/2)


#%% Modulate and recompute
for wg in WG_Array:
    wg.Source(f0)
    # for dipole in wg.DipoleList: dipole.tune(f0,0.8)
    wg.Modulate(k_b,E_b)
    wg.Compute(f0,compute_type='coupled')


# Plot near fields
mlab.init_notebook()
mlab.clf()
for wg in WG_Array:
    wg.MayaviPlot(1,plot_fields=True,field_clip=30)
PlotTools.MayaviAxes(10,-1.5*L,1.5*L,-1.5*L,1.5*L,-L/10,L/2)
mlab.figure(figure=3,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [9]:
#%% Find total far-field and plot
HS.ComputeFarField(f0)

# Far field plotting
directivity=HS.FarField.directivity
k_ff=HS.FarField.k
(theta_grid,phi_grid,th_axis,ph_axis,dth,dph)=math.HalfSphereGrid(HS.FarField.Nth,HS.FarField.Nph)
Dir_dB=10*np.log10(np.real(directivity(k_ff(theta_grid,phi_grid)))+1e-16) 
Dir_dB[Dir_dB<0]=0
rhat=math.SphericalBasis('r')
X=Dir_dB*(rhat.dot(xhat))(theta_grid,phi_grid)
Y=Dir_dB*(rhat.dot(yhat))(theta_grid,phi_grid)
Z=Dir_dB*(rhat.dot(zhat))(theta_grid,phi_grid)

mlab.init_notebook()
mlab.clf()
mlab.mesh(X,Y,Z)
coordmax=np.max([X,Y,Z])
PlotTools.MayaviAxes(10,-coordmax,coordmax,-coordmax,coordmax,0,coordmax)
mlab.figure(figure=4,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

The farfield object in the half-space has more than just fields... it also includes an analysis of the far field, including a report on the beams.  This can be accessed by running,

In [10]:
print(HS.FarField.Beams)

    Directivtiy [dB]  Theta [deg]  Phi [deg]  Side Lobe Level [dB]
0          12.615382    90.000000   0.716216              0.000000
1           9.734571   102.264774  36.045683              2.880811
2           9.721113    51.146626  35.299199              2.894270
3           8.351534  -129.644175  34.622428              4.263848
4           8.254659   -76.551385  36.243040              4.360723
5           1.588918    22.067900  86.489168             11.026465
6           1.502100    19.798876  85.136676             11.113282
7           1.473648   132.466441  88.759413             11.141734
8           1.361545   133.479230  87.623961             11.253837
9           1.337106    23.670538  85.083969             11.278276
10          1.195079   134.492971  87.225095             11.420303
11          1.154143   130.914383  82.965066             11.461239
12          1.121970   129.897835  83.661798             11.493412
13          1.096774    17.525568  85.031816             11.51